# Using callbacks during training
lets you specify a list of objects that Keras will call at the start and end of training, at the start and end of each epoch,and even before and after processing each batch.

In [1]:
# set up
import os
import matplotlib.pyplot as plt
root = ""
chapter_id ="Callbacks"
# path of folder
images_path = os.path.join(root,"images",chapter_id)
os.makedirs(images_path,exist_ok=True)
def save_fig(fig_id,tigh_layout,fig_extension="png",resolution=True):
    # path until file name
    path = os.path.join(images_path,fig_id + "." +fig_extension)
    print("save figure",fig_id)
    if tigh_layout:
        plt.tight_layout()
    plt.savefig(path,fig_extension=fig_extension,resolution=resolution)
 

In [2]:
# load data set
import tensorflow as tf 
import keras
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
print(housing.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

In [3]:
# SPLIT DATA TRAIN FULL AND TESTING 
X_train_full,X_test,y_train_full,y_test = train_test_split(housing['data'],housing['target'],random_state=42)
#SPLIT DATA TRAIN FULL TO TRAIN AND VALIDATION
X_train,X_valid,y_train,y_valid = train_test_split(X_train_full,y_train_full,random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.fit_transform(X_valid)
X_test = scaler.fit_transform(X_test)

# Using callbacks during training

In [4]:
# Set up generate  

keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [5]:
# linear model
model = keras.models.Sequential([
    keras.layers.Dense(30,activation='relu',input_shape=[8]),
    keras.layers.Dense(30,activation='relu'),
    keras.layers.Dense(1)
])

In [6]:
# compile model
model.compile(loss='mse',optimizer=keras.optimizers.SGD(learning_rate=1e-3))
# save best only -> save the best of validation
checkpoint_cb = keras.callbacks.ModelCheckpoint('my_keras_model.h5',save_best_only = True)
history = model.fit(X_train,y_train,epochs=10,
                    validation_data=[X_valid,y_valid],
                    callbacks=[checkpoint_cb])

Epoch 1/10
363/363 [==============================] - 2s 2ms/step - loss: 1.8866 - val_loss: 0.7414
Epoch 2/10
363/363 [==============================] - 0s 991us/step - loss: 0.6577 - val_loss: 0.6221
Epoch 3/10
363/363 [==============================] - 0s 953us/step - loss: 0.5934 - val_loss: 0.5779
Epoch 4/10
363/363 [==============================] - 0s 1ms/step - loss: 0.5557 - val_loss: 0.5572
Epoch 5/10
363/363 [==============================] - 0s 1ms/step - loss: 0.5272 - val_loss: 0.5305
Epoch 6/10
363/363 [==============================] - 0s 1ms/step - loss: 0.5033 - val_loss: 0.5113
Epoch 7/10
363/363 [==============================] - 0s 979us/step - loss: 0.4854 - val_loss: 0.4974
Epoch 8/10
363/363 [==============================] - 0s 1ms/step - loss: 0.4709 - val_loss: 0.4903
Epoch 9/10
363/363 [==============================] - 0s 998us/step - loss: 0.4578 - val_loss: 0.4773
Epoch 10/10
363/363 [==============================] - 0s 980us/step - loss: 0.4474 - val_lo

In [7]:
# rollback to best model
model = keras.models.load_model('my_keras_model.h5')
model.evaluate(X_test,y_test)

162/162 [==============================] - 0s 652us/step - loss: 0.4356


0.4355747401714325

# TRAIN AGAINT 
Train againt but now used the early stopping, thats ill be interupt if the result is not on progress on validation loss function

In [8]:
model.compile(loss='mse',optimizer=keras.optimizers.SGD(learning_rate=1e-3))
#patience : number of epochs with no improvement after which training will be stopped.
"""
Jika tidak ada improvement sebanyak 10 kali pada validatio test on train
maka akan stopped 
"""
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)
history2 = model.fit(X_train,y_train,epochs=100,
                     validation_data=[X_valid,y_valid],
                     callbacks=[checkpoint_cb,early_stopping_cb]) 

Epoch 1/100
363/363 [==============================] - 1s 1ms/step - loss: 0.4393 - val_loss: 0.4655
Epoch 2/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4315 - val_loss: 0.4627
Epoch 3/100
363/363 [==============================] - 0s 992us/step - loss: 0.4259 - val_loss: 0.4562
Epoch 4/100
363/363 [==============================] - 0s 972us/step - loss: 0.4201 - val_loss: 0.4554
Epoch 5/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4154 - val_loss: 0.4506
Epoch 6/100
363/363 [==============================] - 0s 976us/step - loss: 0.4111 - val_loss: 0.4461
Epoch 7/100
363/363 [==============================] - 0s 975us/step - loss: 0.4074 - val_loss: 0.4446
Epoch 8/100
363/363 [==============================] - 0s 942us/step - loss: 0.4040 - val_loss: 0.4434
Epoch 9/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4008 - val_loss: 0.4391
Epoch 10/100
363/363 [==============================] - 0s 999us/step - loss: 0.3

jika di liat bahwa tidak sampai dalam train ke 100 karena tidak akan mengalami progress penurunan gradient pada loss function validation data on training

In [9]:
mse_test2=model.evaluate(X_test,y_test)

162/162 [==============================] - 0s 665us/step - loss: 0.3720


tentunya loss functionnya sangat menurun pada model ini dan akan di simpan pada checkpoint

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                270       
                                                                 
 dense_1 (Dense)             (None, 30)                930       
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 1,231
Trainable params: 1,231
Non-trainable params: 0
_________________________________________________________________


In [11]:
# custom callbacks -> ratio between validation loss and training loss function
class PrintValTrainRatioCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        print("\nVal/Train: {:.2f}".format(logs['val_loss']/ logs['loss']))

In [12]:
# object assigmnt
val_train_ratio_cb = PrintValTrainRatioCallback()
# train
history = model.fit(X_train,y_train,epochs=1,
                    validation_data=[X_valid,y_valid],
                    callbacks=[val_train_ratio_cb])

293/363 [=======================>......] - ETA: 0s - loss: 0.3400
Val/Train: 1.20
363/363 [==============================] - 0s 978us/step - loss: 0.3373 - val_loss: 0.4052


# Using TensorBoard for Visualization

In [13]:
root_logdir = os.path.join(os.curdir,'my_logs')

In [15]:
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir,run_id)


In [16]:
run_logdir = get_run_logdir()
run_logdir

'.\\my_logs\\run_2022_07_07-20_40_53'

In [30]:
checkpoint_cb.best

0.4041360318660736

In [18]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [20]:
model = keras.models.Sequential([
    keras.layers.Dense(30,activation='relu',input_shape=[8]),
    keras.layers.Dense(30,activation='relu'),
    keras.layers.Dense(1)
])
model.compile(loss='mse',optimizer=keras.optimizers.SGD(learning_rate=1e-3))

In [29]:
#log file
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
#current date generated of small function
#'.\\my_logs\\run_2022_07_07-20_40_53'
history3 = model.fit(X_train,y_train,epochs=30,
                     validation_data=(X_valid,y_valid),
                     callbacks=[checkpoint_cb,tensorboard_cb])


Epoch 1/30
363/363 [==============================] - 1s 2ms/step - loss: 1.8866 - val_loss: 0.7414
Epoch 2/30
363/363 [==============================] - 0s 1ms/step - loss: 0.6577 - val_loss: 0.6221
Epoch 3/30
363/363 [==============================] - 0s 1ms/step - loss: 0.5934 - val_loss: 0.5779
Epoch 4/30
363/363 [==============================] - 0s 1ms/step - loss: 0.5557 - val_loss: 0.5572
Epoch 5/30
363/363 [==============================] - 0s 1ms/step - loss: 0.5272 - val_loss: 0.5305
Epoch 6/30
363/363 [==============================] - 0s 1ms/step - loss: 0.5033 - val_loss: 0.5113
Epoch 7/30
363/363 [==============================] - 0s 1ms/step - loss: 0.4854 - val_loss: 0.4974
Epoch 8/30
363/363 [==============================] - 0s 1ms/step - loss: 0.4709 - val_loss: 0.4903
Epoch 9/30
363/363 [==============================] - 0s 1ms/step - loss: 0.4578 - val_loss: 0.4773
Epoch 10/30
363/363 [==============================] - 0s 1ms/step - loss: 0.4474 - val_loss: 0.4696

In [ ]:
%load_ext tensorboard

%tensorboard --logdir=./mylogs --port=6006